# China's COVID-19 situation
*March 29, 2022*

China has been dealing with a surge in COVID-19 cases, and Shanghai is facing another lockdown. Let's do some visualizations for stories on this. First, import pandas and read in Johns Hopkins data.

In [35]:
import pandas as pd

raw = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")

raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/20/22,3/21/22,3/22/22,3/23/22,3/24/22,3/25/22,3/26/22,3/27/22,3/28/22,3/29/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,177093,177191,177255,177321,177321,177321,177321,177520,177602,177658
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,273088,273088,273146,273164,273257,273318,273387,273432,273432,273529
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265550,265562,265573,265585,265599,265612,265621,265629,265641,265651
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,39234,39234,39234,39713,39713,39713,39713,39713,39713,39713
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,99003,99010,99058,99058,99081,99102,99106,99115,99115,99138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,654380,654954,654954,654954,655468,655468,655468,655468,655750,655750
280,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,535,535,535,535,535,535,535,535,535,535
281,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11801,11801,11802,11802,11803,11803,11803,11803,11803,11803
282,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,315623,315892,316088,316190,316312,316422,316501,316550,316550,316550


### China situation

We'll start by reshaping the table to show what we're interested in here: COVID-19 cases in China on the most recent day on record.

In [36]:
china = raw.loc[raw["Country/Region"] == "China", ["Province/State", '3/28/22', '3/29/22']]

Because the JHU data is confirmed cases (cumulative), let's find the new cases from yesterday to today and show it in the same shape.

In [37]:
china["new_cases"] = china["3/29/22"] - china["3/28/22"]

china.loc[:, ["Province/State", "new_cases"]]

,Province/State,new_cases
59,Anhui,3
60,Beijing,3
61,Chongqing,1
62,Fujian,10
63,Gansu,0
64,Guangdong,15
65,Guangxi,9
66,Guizhou,1
67,Hainan,1
68,Hebei,6


I can take this and map it using Datawrapper (check out the result [here](https://www.datawrapper.de/_/E9WZb/)).

### Shanghai situation

Now let's check out the situation in Shanghai over time. We'll start by reshaping the table so that dates are the index. We also drop some rows that came from our transpose.

In [38]:
shanghai = (raw
    .loc[raw["Province/State"] == "Shanghai", :]
    .transpose()
    .drop(["Province/State", "Country/Region", "Lat", "Long"])
    )

shanghai

,84
1/22/20,9
1/23/20,16
1/24/20,20
1/25/20,33
1/26/20,40
...,...
3/25/22,5182
3/26/22,5235
3/27/22,5295
3/28/22,5402


Now we convert the index to datatime.

In [39]:
shanghai.index = pd.to_datetime(shanghai.index)

And calculate the difference (ie. new cases, rather than cumulative cases) and filter by date to show data from Oct. 2021.

In [40]:
shanghai = (shanghai
    .loc[shanghai.index >= "2021-10-01", :]
    .diff()
    .dropna()
)

shanghai

,84
2021-10-02,7
2021-10-03,3
2021-10-04,6
2021-10-05,4
2021-10-06,4
...,...
2022-03-25,47
2022-03-26,53
2022-03-27,60
2022-03-28,107


I also want to add a "rolling" column, because it graphs much more nicely.

In [41]:
shanghai["cases_rolling"] = (shanghai
    .rolling(7).mean()
)

shanghai

,84,cases_rolling
2021-10-02,7,NaN
2021-10-03,3,NaN
2021-10-04,6,NaN
2021-10-05,4,NaN
2021-10-06,4,NaN
...,...,...
2022-03-25,47,31.142857
2022-03-26,53,35.571429
2022-03-27,60,38.285714
2022-03-28,107,48.000000


Let's also change the column names to be more descriptive.

In [42]:
shanghai.columns = ["Daily new cases", "7-day average"]

When you use `.rolling(7)`, the first 7 rows become NaN, so we drop them.

In [43]:
shanghai.dropna().to_clipboard()

And that's it! We [copy the data over to Datawrapper for graphing](https://www.datawrapper.de/_/Elfyq/).

\-30\-